<a href="https://colab.research.google.com/github/Ge0f3/tensorflow_certification/blob/main/Transfer_learning_Flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tf-nightly-gpu
!pip install "tensorflow_hub==0.4.0"
!pip install -U tensorflow_datasets

Requirement already up-to-date: tensorflow_datasets in /usr/local/lib/python3.6/dist-packages (1.0.2)


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt

import tensorflow as tf
tf.enable_eager_execution()

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [ ]:
import logging
import os 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


In [ ]:
#Loading the images 
#Loading the image dataste
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.tgz", 
                                   extract=True)
print(zip_dir)

/root/.keras/datasets/flower_data.zip


In [ ]:
#Training and Testing directory 
base_dir = os.path.join(os.path.dirname(zip_dir), 'flower_photos')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')

In [ ]:
BATCH_SIZE = 100  # Number of training examples to process before updating our models variables
IMG_SHAPE  = 224  # Our training data consists of images with width of 150 pixels and height of 150 pixels
train_image_generator      = ImageDataGenerator(rescale=1./255,
                                               rotation_range=40,
                                                            width_shift_range=0.2,
                                                            height_shift_range=0.2,
                                                            shear_range=0.2,
                                                            zoom_range=0.2,
                                                            horizontal_flip=True,
                                                            fill_mode='nearest')  # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255)  # Generator for our validation data

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE, 
                                                           directory=base_dir, 
                                                           shuffle=True, 
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150) 
                                                           class_mode='binary')

Found 3670 images belonging to 5 classes.


In [ ]:
#loading the state of art neural network 
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"

IMAGE_RES = 224

feature_extractor =  hub.KerasLayer(URL,input_shape = (IMAGE_RES,IMAGE_RES,3))

In [ ]:
feature_extractor

In [ ]:
#Freezing so that the training modeifies only the final layer
feature_extractor.trainable = False

In [ ]:
model = tf.keras.Sequential([
    feature_extractor,
    layers.Dense(5,activation='softmax')
    
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
EPOCHS=15
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=10,
    epochs=EPOCHS,
)

Epoch 1/15
10/10 [==============================] - 18s 2s/step - loss: 1.7815 - acc: 0.2780
Epoch 2/15
10/10 [==============================] - 18s 2s/step - loss: 1.2445 - acc: 0.5490
Epoch 3/15
10/10 [==============================] - 19s 2s/step - loss: 0.9824 - acc: 0.6784
Epoch 4/15
10/10 [==============================] - 18s 2s/step - loss: 0.8388 - acc: 0.7510
Epoch 5/15
10/10 [==============================] - 17s 2s/step - loss: 0.7262 - acc: 0.7730
Epoch 6/15
10/10 [==============================] - 16s 2s/step - loss: 0.6977 - acc: 0.7784
Epoch 7/15
10/10 [==============================] - 16s 2s/step - loss: 0.6438 - acc: 0.8150
Epoch 8/15
10/10 [==============================] - 16s 2s/step - loss: 0.5854 - acc: 0.8440
Epoch 9/15
10/10 [==============================] - 16s 2s/step - loss: 0.6162 - acc: 0.7990
Epoch 10/15
10/10 [==============================] - 17s 2s/step - loss: 0.6020 - acc: 0.8190
Epoch 11/15
10/10 [==============================] - 18s 2s/step - lo

In [ ]:
history.history

{'acc': [0.278,
  0.549,
  0.6783505,
  0.751,
  0.773,
  0.77835053,
  0.815,
  0.844,
  0.799,
  0.819,
  0.824,
  0.84742266,
  0.833,
  0.849,
  0.8546392],
 'loss': [1.7814883828163146,
  1.2445054411888123,
  0.9838162470109684,
  0.838788914680481,
  0.7262226402759552,
  0.696004018955624,
  0.6437859237194061,
  0.5854247272014618,
  0.6161675572395324,
  0.6020499289035797,
  0.5770085334777832,
  0.535695210560081,
  0.5469098448753357,
  0.5399455100297927,
  0.5129684925693827]}

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()

KeyError: ignored